# GPU

In [ ]:
# Runtime / Change runtime type/ GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Found GPU at: /device:GPU:0
Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.7208938249999903
GPU (s):
0.042746929999992744
GPU speedup over CPU: 87x


In [7]:
import cv2
import sys, os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import numpy as np
from numpy import asarray

# DeepFake Detection Algorithm
Input : $D_{real}$, $D_{fake}$ were centered by subtracting the mean of the real training data.


## Create $D_{real}$

In [8]:
# Read images
img_real = []
path = '/content/drive/MyDrive/American_University/2021_Fall/DATA-793-001_Data Science Practicum/Datasets/manipulated_sequences/Deepfakes/raw/videos/data_test'

for root, _, files in os.walk(path):
    current_directory_path = os.path.abspath(root)
    for f in files:
        name, ext = os.path.splitext(f)
        if ext == ".jpg":
            current_image_path = os.path.join(current_directory_path, f)
            current_image = imageio.imread(current_image_path)
            img_real.append(current_image)
img_real = np.array(img_real, dtype=object)
for img in img_real:
  print(img.shape)
print(len(img_real))

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(480, 854, 3)
(480, 854, 3)
10


## Global Centering for each image

In [ ]:
def meanSubtraction(arr):
  new_arr = []
  for i in range(len(arr)):
    img = arr[i]
    #img = img.astype(np.float32) # convert from integers to floats
    img = np.array(img, dtype=np.float32)
    mean = img.mean() # calculate global mean
    img = img - mean # centering of pixels 
    img /= img.std()
    img = [np.round(img, 2) for i in range(len(arr))]
    new_arr.append(img)
  new_arr = np.array(new_arr, dtype=object)
  return new_arr

img_real = meanSubtraction(img_real)
print(len(img_real))
print(img_real[0])

# Preprocessing and data tensor organization

In [ ]:
[U_real, S_real, V_real] = np.linalg.svd(img_real[0])
print(U_real)

In [ ]:
print(U_real.shape)

(720, 1280, 1280)


In [6]:
U_real = []
S_real = []
V_real = []
for i in range(len(img_real)):
  U, S, V = np.linalg.svd(img_real[i], full_matrices=True)
  U_real.append(U)
  S_real.append(S)
  V_real.append(V)
U_real = np.array(U_real)
S_real = np.array(S_real)
V_real = np.array(V_real)


LinAlgError: ignored

# Testing Zone

In [ ]:
"""
img_df = []           
for img in images:
  img = img.flatten()
  img_df.append(img)
  print(img.shape)
img_df = np.array(img_df, dtype=object)
print(img_df)
"""